In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from rasterio import Affine, features
from shapely.geometry import shape
from shapely.ops import cascaded_union
from math import floor, ceil, sqrt
import fiona
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
shapes = []

#with fiona.open("data/humboldt-park/humboldt-park-meters.shp") as hp:
with fiona.open("data/humboldt-park/humboldt-park.geojson") as hp:
    minx, miny, maxx, maxy = hp.bounds
    for f in hp:
        if shape(f['geometry']).is_valid:
            #for c in f['geometry']['coordinates']:
            #    c[0], c[1] = pyproj.transform(inProj, outProj, c[0], c[1])
            #shp = transform(project, shape(f['geometry']))
            shapes.append(shape(f['geometry']))

print len(shapes)

Self-intersection at or near point -87.71170156851403 41.904652090768906


64


In [4]:
max_shape = cascaded_union([shape(s) for s in shapes])
minx, miny, maxx, maxy = max_shape.bounds
dx = dy = 0.00001  # grid resolution; very small because not projected coordinates
lenx = dx * (ceil(maxx / dx) - floor(minx / dx))
leny = dy * (ceil(maxy / dy) - floor(miny / dy))
nx = int(lenx / dx)
ny = int(leny / dy)
gt = Affine(
    dx, 0.0, dx * floor(minx / dx),
    0.0, -dy, dy * ceil(maxy / dy))

print nx 
print ny

12824
4971


In [5]:
'''r = features.rasterize([shapes[0]], (ny, nx), transform=gt)
r2 = features.rasterize([shapes[1]], (ny, nx), transform=gt)
r3 = features.rasterize([shapes[2]], (ny, nx), transform=gt)
#r = np.add(r, r2)

pa = np.zeros((ny, nx), 'd')
pa[r > 0] += 1
print np.amax(pa)
pa[r2 > 0] += 1
print np.amax(pa)
pa[r3 > 0] += 1

with rasterio.drivers():
    #result = features.rasterize([pa], out_shape=(leny, lenx))
    with rasterio.open(
            'test.tif',
            'w',
            driver='GTiff',
            width=ny,
            height=nx,
            count=1,
            dtype=np.uint8,
            nodata=0) as out:
        out.write(pa.astype(np.uint8), indexes=1)'''

"r = features.rasterize([shapes[0]], (ny, nx), transform=gt)\nr2 = features.rasterize([shapes[1]], (ny, nx), transform=gt)\nr3 = features.rasterize([shapes[2]], (ny, nx), transform=gt)\n#r = np.add(r, r2)\n\npa = np.zeros((ny, nx), 'd')\npa[r > 0] += 1\nprint np.amax(pa)\npa[r2 > 0] += 1\nprint np.amax(pa)\npa[r3 > 0] += 1\n\nwith rasterio.drivers():\n    #result = features.rasterize([pa], out_shape=(leny, lenx))\n    with rasterio.open(\n            'test.tif',\n            'w',\n            driver='GTiff',\n            width=ny,\n            height=nx,\n            count=1,\n            dtype=np.uint8,\n            nodata=0) as out:\n        out.write(pa.astype(np.uint8), indexes=1)"

In [6]:
pa = np.zeros((ny, nx), 'd')
for s in shapes:
    r = features.rasterize([s], (ny, nx), transform=gt)
    pa[r > 0] += 1
#pa /= len(shapes)  # normalise values

In [7]:
# Ignore zeros http://stackoverflow.com/questions/22049140/how-can-i-ignore-zeros-when-i-take-the-median-on-columns-of-an-array

print np.amax(pa)
print np.mean(pa)
pa_mask = np.ma.masked_where(pa == 0, pa)
print np.mean(pa_mask)
print np.amin(pa)
print np.amin(pa_mask)


63.0
6.97927015994
9.18404600279
0.0
1.0
0.0


In [8]:
def overlap_mean(neighborhood):
    shapes = []

    with fiona.open("data/{0}/{0}.geojson".format(neighborhood)) as hp:
        for f in hp:
            if shape(f['geometry']).is_valid:
                shapes.append(shape(f['geometry']))
    max_shape = cascaded_union([shape(s) for s in shapes])
    minx, miny, maxx, maxy = max_shape.bounds
    dx = dy = 0.00001  # grid resolution; very small because not projected coordinates
    lenx = dx * (ceil(maxx / dx) - floor(minx / dx))
    leny = dy * (ceil(maxy / dy) - floor(miny / dy))
    nx = int(lenx / dx)
    ny = int(leny / dy)
    gt = Affine(
        dx, 0.0, dx * floor(minx / dx),
        0.0, -dy, dy * ceil(maxy / dy))
    pa = np.zeros((ny, nx), 'd')
    for s in shapes:
        r = features.rasterize([s], (ny, nx), transform=gt)
        pa[r > 0] += 1
    # Mask items that are 0, get mean of those greater
    pa_mask = np.ma.masked_where(pa == 0, pa)
    return np.mean(pa_mask)

print overlap_mean("humboldt-park")

Self-intersection at or near point -87.71170156851403 41.904652090768906


9.18404600279


In [74]:
'''with rasterio.drivers():
    with rasterio.open(
            'mean_humboldt_park.tif',
            'w',
            driver='GTiff',
            width=ny,
            height=ny,
            count=1,
            dtype=np.uint8,
            nodata=0) as out:
        out.write(pa.astype(np.uint8), indexes=1)'''

"with rasterio.drivers():\n    with rasterio.open(\n            'mean_humboldt_park.tif',\n            'w',\n            driver='GTiff',\n            width=ny,\n            height=ny,\n            count=1,\n            dtype=np.uint8,\n            nodata=0) as out:\n        out.write(pa.astype(np.uint8), indexes=1)"

In [75]:
'''
with rasterio.open('data/humboldt-park/humboldt-park.tif') as src:
    r = src.read()
    r_src = src
    profile = src.profile
    indexes = src.indexes
    gt = src.affine
    nx = src.width
    ny = src.height
    
print r_src

pa = np.zeros(r.shape)
for s in shp_shapes:
    rs = features.rasterize([s], out_shape=(ny, nx), transform=gt)
    pa[rs > 0] += 1.0
pa /= len(shp_shapes)  # normalise values
'''

"\nwith rasterio.open('data/humboldt-park/humboldt-park.tif') as src:\n    r = src.read()\n    r_src = src\n    profile = src.profile\n    indexes = src.indexes\n    gt = src.affine\n    nx = src.width\n    ny = src.height\n    \nprint r_src\n\npa = np.zeros(r.shape)\nfor s in shp_shapes:\n    rs = features.rasterize([s], out_shape=(ny, nx), transform=gt)\n    pa[rs > 0] += 1.0\npa /= len(shp_shapes)  # normalise values\n"

In [76]:
#profile = src.profile
'''
profile.update(count=1, compress='lzw')
with rasterio.open('data/mean_humboldt_park.tif', 'w', **profile) as dst:
    dst.write(pa, 1)


from rasterio.transform import IDENTITY

with rasterio.open(
            "data/mean_humboldt_park.tif",
            'w',
            driver='GTiff',
            width=ny,
            height=nx,
            count=1,
            dtype=np.uint8,
            nodata=0,
            transform=IDENTITY) as out:
        out.write(pa.astype(np.uint8))
'''

'\nprofile.update(count=1, compress=\'lzw\')\nwith rasterio.open(\'data/mean_humboldt_park.tif\', \'w\', **profile) as dst:\n    dst.write(pa, 1)\n\n\nfrom rasterio.transform import IDENTITY\n\nwith rasterio.open(\n            "data/mean_humboldt_park.tif",\n            \'w\',\n            driver=\'GTiff\',\n            width=ny,\n            height=nx,\n            count=1,\n            dtype=np.uint8,\n            nodata=0,\n            transform=IDENTITY) as out:\n        out.write(pa.astype(np.uint8))\n'

In [77]:
'''
for s in shapes:
    rs = features.rasterize([s], (ny, nx), transform=gt)
    pa[rs > 0] += 1
    #pa += r
pa /= len(shapes)  # normalise values

with rasterio.drivers():
    result = rasterize([geometry], out_shape=(rows, cols))
    with rasterio.open(
            "test.tif",
            'w',
            driver='GTiff',
            width=cols,
            height=rows,
            count=1,
            dtype=numpy.uint8,
            nodata=0,
            transform=IDENTITY as out:
        out.write(result.astype(numpy.uint8), indexes=1)
'''

'\nfor s in shapes:\n    rs = features.rasterize([s], (ny, nx), transform=gt)\n    pa[rs > 0] += 1\n    #pa += r\npa /= len(shapes)  # normalise values\n\nwith rasterio.drivers():\n    result = rasterize([geometry], out_shape=(rows, cols))\n    with rasterio.open(\n            "test.tif",\n            \'w\',\n            driver=\'GTiff\',\n            width=cols,\n            height=rows,\n            count=1,\n            dtype=numpy.uint8,\n            nodata=0,\n            transform=IDENTITY as out:\n        out.write(result.astype(numpy.uint8), indexes=1)\n'